In [1]:
import json
from tqdm import tqdm
from openai import OpenAI
import re
import time

In [ ]:
def virtual_patient(msgs):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=msgs,
        model="deepseek-v3",
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion

In [ ]:
def dialogue_monitor(msgs):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=msgs,
        model="deepseek-v3",
        max_tokens=10,
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion

In [ ]:
def doctor_o3(msgs):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=msgs,
        model="o3",
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion

In [5]:
def gen_group_chat_prompt(group_chat):
    prompt = ""
    for data in group_chat:
        prompt += f"{data['role']}:{data['content']}\n"
    return prompt

In [6]:
def gen_vp_prompt(vp_info):
    prompt_list = []
    for question in vp_info['question_list']:
        prompt_list.append([vp_info['ehr_prompt'], "\n患者核心诉求：" + json.dumps(question, ensure_ascii=False)])
    return prompt_list

In [7]:
vp_info = json.load(open('../../dataset/vp_info.json', 'r'))
vp_prompt_list = []
for data in vp_info:
    vp_prompt_list.extend(gen_vp_prompt(data))

In [8]:
group_chat = json.load(open('../../dataset/group_chat.json', 'r'))
group_chat_prompt = gen_group_chat_prompt(group_chat[:1000])

In [9]:
def get_recode(role, response, inference_time):
    return {
        "role": role,
        "content": response.choices[0].message.content,
        "completion_tokens": response.usage.completion_tokens,
        "prompt_tokens": response.usage.prompt_tokens,
        "inference_time": inference_time
    }

In [18]:
# recoder_list = []
recoder_list = json.load(open('../../output/o3_conv.json', 'r'))

for vp_prompt in tqdm(vp_prompt_list[len(recoder_list):]):
    recoder = {
        "vp_prompt": vp_prompt[0] + vp_prompt[1],
        "conversation": []
    }

    dialogue_monitor_msgs = [{
        "role": "system",
        "content": "你是一名对话内容审查员，功能是判断对话是否已经完成，你只需要输出True或False即可"
    },
    {
        "role": "user",
        "content": vp_prompt[1] + "请根据**患者的核心诉求**，判断下面对话中患者是否已经获取到了所有需要的信息，如果是则输出True，否则输出False，不需要输出其他内容。"
    },
    {
        "role": "user",
        "content": "对话内容："
    }]

    virtual_patient_msgs = [{
        "role": "system",
        "content": "现在你是一名乳腺癌术后患者，你的患者信息和核心诉求在上述内容中已被明确定义。目前的场景为你正在群聊当中向医护人员寻求帮助。"
    },
    {
        "role": "user",
        "content": f"乳腺癌术后康复群群聊示例：\n{group_chat_prompt}\n\n" + vp_prompt[0] + vp_prompt[1] + "现在你是一名乳腺癌术后患者，你的患者信息和核心诉求在上述内容中已被明确定义。目前的场景为你正在群聊当中向医护人员寻求帮助。请仿照'乳腺癌术后康复群群聊示例'中患者交流的方式，表达你的诉求。注意：每次只需要说出一个诉求，不一定要按顺序说出诉求，对话过程注意关联上下文和你的个人信息，尽可能能够对我说的话作出一些反馈。在满足核心诉求后可以礼貌结束对话。我将作为医生与你对话，你只需要扮演患者即可。"
    }
    ]

    doctor_msgs = [{
        "role": "system",
        "content": "现在你是一名乳腺癌专科医师，你的患者在手术后联系你获取帮助，该患者的信息如下：" + vp_prompt[0] + "\n请结合患者信息给出尽可能专业有效并且容易理解的反馈。"
    }]

    count = 0

    while 1:
        # generate virtual patient's content
        start_time = time.time()
        vp_response = virtual_patient(virtual_patient_msgs)
        end_time = time.time()
        vp_content = vp_response.choices[0].message.content
        recoder['conversation'].append(get_recode("virtual patient", vp_response, end_time-start_time))
        
        dialogue_monitor_msgs[2]['content'] += f"患者：{vp_content}"
        virtual_patient_msgs.append({
            "role": "assistant",
            "content": vp_content
        })
        doctor_msgs.append({
            "role": "user",
            "content": vp_content
        })

        # generate doctor's content
        start_time = time.time()
        doctor_response = doctor_o3(doctor_msgs)
        end_time = time.time()
        doctor_content = re.sub(r"> Reasoning.*?Reasoned for a couple of seconds", "", doctor_response.choices[0].message.content, flags=re.DOTALL)
        recoder['conversation'].append(get_recode("doctor", doctor_response, end_time-start_time))

        dialogue_monitor_msgs[2]['content'] += f"医生：{doctor_content}"
        virtual_patient_msgs.append({
            "role": "user",
            "content": doctor_content
        })
        doctor_msgs.append({
            "role": "assistant",
            "content": doctor_content
        })

        # decide whether stop conversation
        monitor_response = dialogue_monitor(dialogue_monitor_msgs)
        monitor_content = monitor_response.choices[0].message.content
        if 'True' in monitor_content or 'true' in monitor_content:
            recoder_list.append(recoder)
            break
        else:
            count += 1
            if count > 6:
                print("Too many turns. Force to stop!")
                recoder_list.append(recoder)
                break

  0%|          | 0/52 [00:00<?, ?it/s]

  2%|▏         | 1/52 [05:16<4:29:14, 316.75s/it]

Too many turns. Force to stop!


  4%|▍         | 2/52 [09:37<3:56:38, 283.96s/it]

Too many turns. Force to stop!


...

 92%|█████████▏| 48/52 [3:09:01<14:19, 214.92s/it]  

Too many turns. Force to stop!


 96%|█████████▌| 50/52 [3:16:50<07:37, 228.85s/it]

Too many turns. Force to stop!


100%|██████████| 52/52 [3:25:38<00:00, 237.28s/it]


In [19]:
json.dump(recoder_list, open('../../output/o3_conv.json', 'w'), ensure_ascii=False)

In [20]:
json.dump(recoder_list, open('../../output/o3_conv_raw.json', 'w'), ensure_ascii=False)

In [21]:
print("您好！您描述的术后伤口紧绷感和发麻是术后常见现象，目前无需过度担心，但需注意科学护理和观察。**\n\n### 一、症状原因解析\n1. **紧绷感**：  \n   - 与手术创面愈合直接相关。皮肤和深层组织修复时会产生瘢痕挛缩，一般术后3-6个月逐渐缓解。  \n   - *您的病理显示为IA期，术中清扫35枚淋巴结且无转移，说明手术范围较大，紧绷感可能更明显。*\n\n2. **发麻/针刺感**：  \n   - 术中可能影响胸壁或腋窝的皮神经（如肋间臂神经），导致暂时性感觉异常。  \n   - *您未出现脉管癌栓或神经侵犯，此症状多为可逆性，多数在3-12个月内改善。*\n\n---\n\n### 二、专业护理建议\n1. **伤口管理**  \n   - ✔️ **保持干燥清洁**：淋浴后用无菌纱布轻拍干燥，避免用力擦拭。  \n   - ✔️ **观察警示征象**：若出现红肿、渗液、发热或剧痛，需立即联系医生。  \n   - ❌ **避免刺激**：3个月内勿在伤口使用精油、酒精或过热敷贴。\n\n2. **缓解紧绷感**  \n   - 穿戴医用弹力胸衣（需主刀医生确认松紧度）。  \n   - 术后4周起可轻柔按摩瘢痕（沿伤口边缘由内向外打圈，每次5分钟，2次/日）。\n\n3. **神经恢复策略**  \n   - 口服甲钴胺（维生素B12衍生物）营养神经（需经主刀医生处方）。  \n   - 术后6周开始低频电刺激治疗（需康复科评估）。\n\n---\n\n### 三、康复时间轴与随访\n- **术后1-2周**：以肩关节前屈<90°的被动活动为主  \n- **术后3-6周**：逐步增加爬墙训练和梳头动作  \n- **术后3个月**：评估瘢痕挛缩情况，必要时使用硅酮凝胶辅料  \n- **术后6个月**：复查乳腺彩超时同步评估感觉恢复进展  \n\n---\n\n### 四、需警惕的特殊情况\n若出现以下情况请立即就医：  \n🔴 伤口突然渗液伴异味  \n🔴 上肢持续性肿胀或皮肤发红  \n🔴 麻木范围扩大至前臂  \n（您已完成腋窝淋巴结清扫，此类症状需排除淋巴水肿）\n\n---\n\n### 五、后续治疗衔接提醒\n1. **靶向治疗必要性**：  \n   您为Her2阳性（3+或FISH阳性），指南推荐术后1年内完成曲妥珠单抗靶向治疗。请与主治医生确认是否需补充该方案。\n\n2. **骨健康管理**：  \n   来曲唑可能导致骨密度下降，建议：  \n   - 每6个月检测维生素D水平  \n   - 每日补充钙剂600mg+维生素D3 800IU  \n   - 半年后行双能X线骨密度检测（DXA）\n\n---\n\n**建议本周返院复查时：**  \n① 请主刀医生触诊评估瘢痕柔韧性  \n② 开具乳腺癌术后专用功能评估量表  \n③ 预约康复科制定个性化训练计划  \n\n祝您康复顺利！如有其他不适请随时联系。\n\n--- \n\n**注：** 此回复基于国际乳腺癌指南（NCCN/ESMO）及中国抗癌协会乳腺癌诊疗规范，具体执行需经主刀医生确认。")

您好！您描述的术后伤口紧绷感和发麻是术后常见现象，目前无需过度担心，但需注意科学护理和观察。**

### 一、症状原因解析
1. **紧绷感**：  
   - 与手术创面愈合直接相关。皮肤和深层组织修复时会产生瘢痕挛缩，一般术后3-6个月逐渐缓解。  
   - *您的病理显示为IA期，术中清扫35枚淋巴结且无转移，说明手术范围较大，紧绷感可能更明显。*

2. **发麻/针刺感**：  
   - 术中可能影响胸壁或腋窝的皮神经（如肋间臂神经），导致暂时性感觉异常。  
   - *您未出现脉管癌栓或神经侵犯，此症状多为可逆性，多数在3-12个月内改善。*

---

### 二、专业护理建议
1. **伤口管理**  
   - ✔️ **保持干燥清洁**：淋浴后用无菌纱布轻拍干燥，避免用力擦拭。  
   - ✔️ **观察警示征象**：若出现红肿、渗液、发热或剧痛，需立即联系医生。  
   - ❌ **避免刺激**：3个月内勿在伤口使用精油、酒精或过热敷贴。

2. **缓解紧绷感**  
   - 穿戴医用弹力胸衣（需主刀医生确认松紧度）。  
   - 术后4周起可轻柔按摩瘢痕（沿伤口边缘由内向外打圈，每次5分钟，2次/日）。

3. **神经恢复策略**  
   - 口服甲钴胺（维生素B12衍生物）营养神经（需经主刀医生处方）。  
   - 术后6周开始低频电刺激治疗（需康复科评估）。

---

### 三、康复时间轴与随访
- **术后1-2周**：以肩关节前屈<90°的被动活动为主  
- **术后3-6周**：逐步增加爬墙训练和梳头动作  
- **术后3个月**：评估瘢痕挛缩情况，必要时使用硅酮凝胶辅料  
- **术后6个月**：复查乳腺彩超时同步评估感觉恢复进展  

---

### 四、需警惕的特殊情况
若出现以下情况请立即就医：  
🔴 伤口突然渗液伴异味  
🔴 上肢持续性肿胀或皮肤发红  
🔴 麻木范围扩大至前臂  
（您已完成腋窝淋巴结清扫，此类症状需排除淋巴水肿）

---

### 五、后续治疗衔接提醒
1. **靶向治疗必要性**：  
   您为Her2阳性（3+或FISH阳性），指南推荐术后1年内完成曲妥珠单抗靶向治疗。请与主治医生确认是否需补充该方案。

2. **骨健康管理**：  
   来曲唑可能导